In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
dbutils.fs.ls("abfss://landing@salesmarketingstorage1.dfs.core.windows.net")

In [0]:
df_customers = spark.read \
.options(header=True) \
.csv("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/customers")

df_customers.show()

In [0]:
schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("customer_unique_id", StringType(), True),
    StructField("customer_zip_code_prefix", StringType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_state", StringType(), True),
])

In [0]:
df_customers = (
    spark.readStream 
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("mode", "PERMISSIVE")
    .schema(schema)
    .load("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/archive/olist_customers_dataset.csv")
    .writeStream
    .format("delta")
    .option("schemaLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/bronze/customers")
    .option("checkpointLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/customers") 
    .table("bronze.customers")
)

In [0]:
dbutils.fs.rm("abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/customers", True)


In [0]:
for stream in spark.streams.active:
    print(stream.name, stream.isActive, stream.lastProgress)

In [0]:
%sql
DESCRIBE HISTORY sales_marketing_ws.bronze.customers;